In [2]:
import google.generativeai as genai
import yaml
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)
GEMINI_API_KEY = config['GEMINI_API_KEY']
genai.configure(api_key=GEMINI_API_KEY)

In [3]:
model = genai.GenerativeModel('gemini-2.0-flash-exp')

In [ ]:
pip install pypdf

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
from pypdf import PdfReader

reader = PdfReader("Resume-SoumyaPandey.pdf")
number_of_pages = len(reader.pages)
page = reader.pages[0]
text = page.extract_text()
print(text)

Soumya Pandey
+91 8169751711 | E‐mail GitHub | LinkedIn
Education
B.Techin Data Science, Economics & Business ‐ Plaksha University, Mohali CGPA: 8.27 2027
12th IBDP‐ S.M.Shetty International, Mumbai Grade: 38/45 2023
10th IGCSE‐ S.M.Shetty International, Mumbai Percentage: 94.3% 2021
Work Experience
Graphic Design and Machine Learning Intern, Chayan.AI May 2024 – Jul 2024
• Built a simple chatbot to guide students in creating resumes. Gained hands‐on experience in data processing and natural language understanding,
applying core Machine Learning concepts.
• Designed and implemented a React.js interface, enhancing user experience (UX) through iterative testing and feedback.
• Created wireframes in Figma to illustrate the web application’s layout, streamlining the design‐to‐development workflow.
Student Tutor, Office of Global Engagements, Plaksha University March 2025 ‐ Present
• Research global trends, international partnerships, and cultural dynamics to support decision‐making within 

In [5]:
import textwrap
import json

def generate_prompt_for_resume_analysis(resume_text):
    # Build the prompt to send to your LLM:
    return textwrap.dedent(f"""
    You are an expert resume evaluator. Given the following resume text, analyze it thoroughly to:

    1. Score the following skills on a scale of 1 to 100:
        - Communication
        - Technical Skills
        - Creativity
        - Leadership
        - Problem Solving
    2. Determine the person's top **strengths** based on these scores.
    3. Identify the person's **weaknesses** or areas for improvement.
    4. Google search for ways to strengthen their weaknesses, and recommend specific courses/action items they can do to improve that score.
    5. Provide a summary of the resume in 3-4 sentences, highlighting key achievements and experiences.

    Respond in the following JSON format:
    {{
        "skills": {{
            "Communication": int,
            "Technical Skills": int,
            "Creativity": int,
            "Leadership": int,
            "Problem Solving": int
        }},
        "strengths": [list of strings],
        "weaknesses": [list of strings],
        "action items": [list of strings],
        "summary": "string"
    }}

    Resume Text:
    \"\"\"{resume_text}\"\"\"
    """)

def analyze_resume_with_gemini(resume_text, model_name="models/gemini-pro"):
    """
    This function is supposed to call your LLM (Gemini), but here we provide a
    placeholder for demonstration purposes. Replace the dummy response with
    your actual model call if you have the 'genai' library installed.
    """
    # Generate the prompt:
    prompt = generate_prompt_for_resume_analysis(resume_text)

    # --------------------------------------------------------------------
    # If you have genai and the gemini model set up, you’d do something like:
    #
    # import genai
    # model = genai.GenerativeModel(model_name)
    # response = model.generate_content(prompt)
    # return response.text
    #
    # --------------------------------------------------------------------

    # For now, we'll just return a dummy JSON string:
    dummy_json_response = {
        "skills": {
            "Communication": 88,
            "Technical Skills": 92,
            "Creativity": 76,
            "Leadership": 85,
            "Problem Solving": 80
        },
        "strengths": ["Technical Skills", "Communication", "Leadership"],
        "weaknesses": ["Creativity", "Problem Solving"],
        "action items": [
            "Enroll in a creative thinking course",
            "Practice systematic problem-solving exercises"
        ],
        "summary": (
            "This candidate has a strong technical background and demonstrated leadership ability. "
            "They excel at communicating and delivering high-quality work. Creativity and problem-solving "
            "skills can be further refined to unlock their full potential."
        )
    }
    return json.dumps(dummy_json_response)

# Provide some sample resume text
text = """
John Doe
Senior Software Engineer with 10 years of experience in backend development.
Skilled in Python, C++, and cloud architecture. Led small teams to deliver complex projects.
Excellent communicator with strong attention to detail and problem-solving abilities.
Interested in exploring creative solutions to modern technology challenges.
"""

# 1. Generate analysis
result = analyze_resume_with_gemini(text)
print("Raw JSON response from model (or placeholder):")
print(result)

# 2. Write to file
with open('resume_analysis.json', 'w') as f:
    f.write(result)

# 3. Read JSON file back in
with open('resume_analysis.json', 'r') as f:
    analysis_result = json.load(f)

# 4. Verify JSON content
print("\nLoaded JSON as Python dictionary:")
print(analysis_result)


Raw JSON response from model (or placeholder):
{"skills": {"Communication": 88, "Technical Skills": 92, "Creativity": 76, "Leadership": 85, "Problem Solving": 80}, "strengths": ["Technical Skills", "Communication", "Leadership"], "weaknesses": ["Creativity", "Problem Solving"], "action items": ["Enroll in a creative thinking course", "Practice systematic problem-solving exercises"], "summary": "This candidate has a strong technical background and demonstrated leadership ability. They excel at communicating and delivering high-quality work. Creativity and problem-solving skills can be further refined to unlock their full potential."}

Loaded JSON as Python dictionary:
{'skills': {'Communication': 88, 'Technical Skills': 92, 'Creativity': 76, 'Leadership': 85, 'Problem Solving': 80}, 'strengths': ['Technical Skills', 'Communication', 'Leadership'], 'weaknesses': ['Creativity', 'Problem Solving'], 'action items': ['Enroll in a creative thinking course', 'Practice systematic problem-solvin

In [47]:
import json
import textwrap

# -------------------------------------------------------
# 1. Simulated "Google Search" tool
# -------------------------------------------------------
def google_search(query):
    """
    This simulates a Google search. In a real scenario, you would integrate
    an API call or a library that performs actual web searches. 
    """
    print(f"[Tool google_search] Received query: {query}")
    # For demonstration, return a static mock result:
    return [
        f"Top 1 result for '{query}': A recommended online course from example.com",
        f"Top 2 result for '{query}': Another recommended online course from example.org"
    ]

# -------------------------------------------------------
# 2. Generate ReAct-Style Prompt
# -------------------------------------------------------
def generate_react_prompt(resume_text):
    """
    Constructs a prompt instructing the LLM to:
      - reason step by step
      - call the 'google_search' tool whenever it needs external information
      - produce the final JSON as requested
    """
    # We embed the ReAct instructions within the system prompt.
    prompt = textwrap.dedent(f"""
    You are an expert resume evaluator using the ReAct framework. 
    You can use the following tool(s) when needed:
    
    1) google_search

    When you need more information about ways to improve a particular skill or find courses, 
    you can call the tool like this:
      Action: google_search
      Action Input: <the query>

    The tool will respond with some search results. Then you incorporate those into your 
    final reasoning. You must *only* use the tool if you truly need it. 
    
    Here is the final task:

    1. Score the following skills on a scale of 1 to 100:
       - Communication
       - Technical Skills
       - Creativity
       - Leadership
       - Problem Solving

    2. Determine the person's top **strengths** based on these scores.
    3. Identify the person's **weaknesses** or areas for improvement.
    4. Use the google_search tool to suggest specific courses or action items to strengthen their weaknesses.
    5. Provide a summary of the resume in 3-4 sentences, highlighting key achievements and experiences.

    You must return the final answer in valid JSON with the structure:
    {{
        "skills": {{
            "Communication": int,
            "Technical Skills": int,
            "Creativity": int,
            "Leadership": int,
            "Problem Solving": int
        }},
        "strengths": [list of strings],
        "weaknesses": [list of strings],
        "action items": [list of strings],
        "summary": "string"
    }}

    Resume Text:
    \"\"\"{resume_text}\"\"\"

    Follow this step-by-step approach:
    1. Think about the resume content and skill assessment.
    2. Decide if you need to call a tool to gather course recommendations (only if needed).
    3. Provide a final JSON answer. Do not include any extra keys.

    Remember, output *only* the final JSON. 

    Begin your step-by-step reasoning now:
    """)
    return prompt

# -------------------------------------------------------
# 3. Mock LLM ReAct chain
# -------------------------------------------------------
def react_resume_analysis(resume_text):
    """
    A simulated ReAct chain that:
    - Creates a ReAct prompt
    - "Parses" the LLM's step-by-step reasoning
    - Calls the google_search tool if needed
    - Returns a final JSON response
    """

    # 3.1 Generate the ReAct prompt
    prompt = generate_react_prompt(resume_text)
    print("[DEBUG] Prompt to LLM:\n", prompt)

    # 3.2 Simulate an LLM that “thinks” and might call google_search
    #     We'll pretend the LLM wrote out a chain of thought like:
    #
    # """
    # Thoughts: The candidate is strong in Technical Skills, Communication, and Leadership.
    # Their weaknesses might be Creativity and Problem Solving...
    # Action: google_search
    # Action Input: "Best online courses for improving creativity and problem solving"
    # Observation: ["Top 1...example.com", "Top 2...example.org"]
    # Final Answer (JSON)
    # """
    #
    # We'll simulate that entire chain below.

    # Step 1: LLM decides on skill scores:
    skill_scores = {
        "Communication": 85,
        "Technical Skills": 90,
        "Creativity": 70,
        "Leadership": 88,
        "Problem Solving": 75
    }

    # Step 2: LLM decides to do a google_search for improvement courses:
    query = "Online courses to improve creativity and problem solving"
    search_results = google_search(query)  # "Action"
    # The LLM would parse these results into the final recommendation.

    # Step 3: LLM final reasoning & JSON response
    # In a real ReAct loop, the LLM would incorporate the "Observation" from the tool 
    # into the final answer. Here, we just emulate it:
    final_json = {
        "skills": skill_scores,
        "strengths": [
            "Technical Skills",
            "Leadership",
            "Communication"
        ],
        "weaknesses": [
            "Creativity",
            "Problem Solving"
        ],
        "action items": [
            "Complete an online course focusing on design thinking.",
            "Enroll in a structured problem-solving workshop.",
            "Practice idea-generation exercises from example.org."
        ],
        "summary": (
            "This candidate has a solid background in software engineering with excellent "
            "technical and leadership capabilities. They communicate effectively and have "
            "delivered results on complex projects. Improving creativity and systematic "
            "problem solving could further enhance their overall impact."
        )
    }

    return json.dumps(final_json, indent=2)

# -------------------------------------------------------
# 4. Running the Analysis
# -------------------------------------------------------
if __name__ == "__main__":
    # Example Resume Text
    resume_text = """
    John Doe
    Senior Software Engineer with 10 years of experience in backend development.
    Skilled in Python, C++, and cloud architecture. Led small teams to deliver
    complex projects on schedule. Interested in exploring creative solutions 
    to modern tech challenges.
    """

    # Perform the ReAct-based resume analysis
    result_json_str = react_resume_analysis(resume_text)

    # Print the final JSON
    print("\n[FINAL RESPONSE]\n", result_json_str)

    # Optionally, write to a file or parse it into a Python dictionary:
    analysis_result = json.loads(result_json_str)
    print("\n[PARSED JSON]\n", analysis_result)


[DEBUG] Prompt to LLM:
 
You are an expert resume evaluator using the ReAct framework. 
You can use the following tool(s) when needed:

1) google_search

When you need more information about ways to improve a particular skill or find courses, 
you can call the tool like this:
  Action: google_search
  Action Input: <the query>

The tool will respond with some search results. Then you incorporate those into your 
final reasoning. You must *only* use the tool if you truly need it. 

Here is the final task:

1. Score the following skills on a scale of 1 to 100:
   - Communication
   - Technical Skills
   - Creativity
   - Leadership
   - Problem Solving

2. Determine the person's top **strengths** based on these scores.
3. Identify the person's **weaknesses** or areas for improvement.
4. Use the google_search tool to suggest specific courses or action items to strengthen their weaknesses.
5. Provide a summary of the resume in 3-4 sentences, highlighting key achievements and experiences.



In [48]:
from fastapi import FastAPI, Body
import uvicorn
import json
import textwrap
from typing import List, Dict

app = FastAPI()

# -------------------------------------------------------
# 1. Simulated "Google Search" tool
# -------------------------------------------------------
def google_search(query: str) -> List[str]:
    """
    This simulates a Google search. In a real scenario, you would integrate
    an API or library that performs actual web searches.
    """
    print(f"[Tool google_search] Received query: {query}")
    # Return a mock result for demonstration:
    return [
        f"Result 1 for '{query}': A recommended online course from example.com",
        f"Result 2 for '{query}': Another recommended online course from example.org"
    ]

# -------------------------------------------------------
# 2. Generate ReAct-Style Prompt
# -------------------------------------------------------
def generate_react_prompt(resume_text: str) -> str:
    """
    Constructs a prompt instructing the LLM to:
      - reason step by step
      - call the 'google_search' tool when needed
      - produce the final JSON as requested
    """
    prompt = textwrap.dedent(f"""
    You are an expert resume evaluator using the ReAct framework. 
    You can use the following tool(s):
    
    1) google_search

    When you need more information about ways to improve a particular skill or find courses, 
    you can call the tool like this:
      Action: google_search
      Action Input: <the query>

    The tool will respond with some search results. Then you incorporate those into your 
    final reasoning. You must *only* use the tool if you truly need it. 
    
    Task:
    1. Score these skills from 1 to 100: Communication, Technical Skills, Creativity, Leadership, Problem Solving.
    2. Determine top strengths.
    3. Identify weaknesses.
    4. Use the google_search tool for improvement recommendations.
    5. Return a summary of the resume in 3-4 sentences.

    Return the final answer in valid JSON:
    {{
        "skills": {{
            "Communication": int,
            "Technical Skills": int,
            "Creativity": int,
            "Leadership": int,
            "Problem Solving": int
        }},
        "strengths": [list of strings],
        "weaknesses": [list of strings],
        "action items": [list of strings],
        "summary": "string"
    }}

    Resume Text:
    \"\"\"{resume_text}\"\"\"

    Begin your step-by-step reasoning now:
    """)
    return prompt

# -------------------------------------------------------
# 3. Mock LLM ReAct chain
# -------------------------------------------------------
def react_resume_analysis(resume_text: str) -> str:
    """
    A simulated ReAct chain:
      - Creates a ReAct prompt
      - "Parses" the LLM's step-by-step reasoning
      - Calls the google_search tool if needed
      - Returns a final JSON response (as a string)
    """
    prompt = generate_react_prompt(resume_text)
    print("[DEBUG] Prompt to LLM:\n", prompt)

    # In a real system:
    # 1) Send `prompt` to an LLM,
    # 2) Parse out any "Action: google_search" calls,
    # 3) Execute them, feed back "Observation",
    # 4) Then capture the LLM's final JSON answer.

    # For demonstration, we'll just return a static JSON.

    # Simulated skill scores
    skill_scores = {
        "Communication": 85,
        "Technical Skills": 90,
        "Creativity": 70,
        "Leadership": 88,
        "Problem Solving": 75
    }

    # Simulate the LLM deciding to run google_search for improvements
    query = "Online courses to improve creativity and problem solving"
    search_results = google_search(query)  # "Action"

    # Incorporate the tool output into an action item (fake example)
    action_items = [
        "Complete an online course focusing on design thinking.",
        "Enroll in a structured problem-solving workshop."
    ]
    # Perhaps we add one search result as an example:
    action_items.append(search_results[0])

    final_json = {
        "skills": skill_scores,
        "strengths": ["Technical Skills", "Leadership", "Communication"],
        "weaknesses": ["Creativity", "Problem Solving"],
        "action items": action_items,
        "summary": (
            "This candidate has a solid background in software engineering with "
            "excellent technical and leadership capabilities. They communicate "
            "effectively and have delivered results on complex projects. "
            "Improving creativity and systematic problem-solving could further "
            "enhance their overall impact."
        )
    }

    return json.dumps(final_json, indent=2)

# -------------------------------------------------------
# 4. API Endpoint
# -------------------------------------------------------
@app.post("/analyze_resume")
def analyze_resume_endpoint(resume_text: str = Body(..., embed=True)) -> Dict:
    """
    Analyzes a resume using a ReAct-style approach and returns JSON
    with skill scores, strengths, weaknesses, action items, and a summary.
    
    The request body must be in the form:
    {
       "resume_text": "the resume content here"
    }
    """
    result_json_str = react_resume_analysis(resume_text)
    result_dict = json.loads(result_json_str)
    return result_dict

# -------------------------------------------------------
# 5. Run the server (if you run this script directly)
# -------------------------------------------------------
# To run:
#   uvicorn your_script_name:app --reload
# Then test with a POST request to http://127.0.0.1:8000/analyze_resume
# Sample cURL:
#   curl -X POST -H "Content-Type: application/json" \\
#   -d '{"resume_text": "John Doe ... Senior Developer ..."}' \\
#   http://127.0.0.1:8000/analyze_resume
# 
# or from a REST client (Postman, etc.).
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)


RuntimeError: asyncio.run() cannot be called from a running event loop

In [1]:
#json parsing

import re

response_text = response.text.strip()
if response_text.startswith("```json"):
    response_text = re.sub(r"```json|```", "", response_text).strip()

try:
    parsed = json.loads(response_text)
except json.JSONDecodeError:
    fallback = {"summary": response_text, "skills": {}, "strengths": []}


NameError: name 'response' is not defined

In [2]:
from time import perf_counter

def run_and_benchmark(model, prompt):
    t0 = perf_counter()
    response = model.generate_content(prompt)
    duration = perf_counter() - t0

    try:
        parsed = json.loads(response.text)
        return parsed, duration
    except:
        return {"summary": response.text}, duration


In [3]:
agent_plan = {
    "nodes": {
        "parse_resume": parse_resume_node,
        "parse_job": parse_job_criteria_node,
        "compare": match_score_node,
        "generate_fixes": action_suggestions_node,
        "interview": interview_q_node
    },
    "edges": {
        "parse_resume": ["parse_job"],
        "parse_job": ["compare"],
        "compare": ["generate_fixes", "interview"]
    }
}


NameError: name 'parse_resume_node' is not defined

In [5]:
# each node is basially a function
def compare_skills_node(state):
    resume_skills = set(state["resume"]["skills"])
    job_skills = set(state["job"]["requirements"])
    matched = resume_skills & job_skills
    score = len(matched) / len(job_skills)

    return {
        "match_score": round(score * 100, 2),
        "matched_skills": list(matched)
    }


In [10]:
import time
import google.api_core.exceptions

for retry in range(3):
    try:
        response = model.generate_content(prompt, generation_config={
            "max_output_tokens": 1024
        })
        break
    except google.api_core.exceptions.InvalidArgument as e:
        if "exceeds the token limit" in str(e):
            print("⚠️ Token limit exceeded, trimming prompt...")
            prompt = prompt[:3000]  # manually truncate
            time.sleep(2 * retry)
        else:
            raise e



SyntaxError: 'return' outside function (1787601338.py, line 6)

In [8]:
prompt = f"""
Only extract and score skills explicitly mentioned in the resume below.

Resume:
\"\"\"{resume_text}\"\"\"

Return JSON like:
{{
  "skills": {{
    "Python": 85,
    "C++": 90
  }}
}}
"""


NameError: name 'resume_text' is not defined

In [9]:
rag_context = retrieve_related_projects(resume_text)
prompt = f"""
Given the resume and project context, evaluate these skills.

Context:
\"\"\"{rag_context}\"\"\"

Resume:
\"\"\"{resume_text}\"\"\"
"""


NameError: name 'retrieve_related_projects' is not defined